In [2]:
# !pip install -U -q PyDrive
# import pandas as pd
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# # Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [3]:
def fixPad(item):
  length = len(str(item))
  if length == 6:
    return str(item)
  elif length == 5:
    return '0' + str(item)
  elif length == 4:
    return '00' + str(item)
  else:
    return '000' + str(item)
    # length = len(str(item))
    # if length == 6:
    #     return item
    # else:
    #   return fixPad('0' + str(item))

def formatHourly(row):
    if (row['timeOfDay'] == 'PM'):
        return int(row['time'])*2
    else:
        return int(row['time'])
    
def formatTime(item):
    if item!= 0:
        return int(item/100)
    else:
        return int(item)
    
def formatDate(time):
    year = time[:4]
    month = time[5:7]
    day = time[8:10]
    return month + '/' + day + '/' + year

## **Import Ride Count**

In [4]:
# # ride_url = 'https://drive.google.com/open?id=1SSERtxHig9XPu35ZTxBx5YXSSf_s3wbP'
# id = '1SSERtxHig9XPu35ZTxBx5YXSSf_s3wbP' # The shareable link
# downloaded = drive.CreateFile({'id':id}) 
# downloaded.GetContentFile('Filename.csv')  
# ride_info = pd.read_csv('Filename.csv')
ride_info = pd.read_csv('../masterNoDemo.csv')

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
ride_info.head()

,Trip ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,...,Shared Trip Authorized,Trips Pooled,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,hour,date
0,c2fc4879596363b97b4cc660aab548f28db0aa71,2019-01-26 20:15:00,01/26/2019 08:30:00 PM,600.0,1.8,1.703106e+10,1.703106e+10,6.0,6.0,5.0,...,False,1.0,41.949061,-87.661643,POINT (-87.6616429039 41.9490605256),41.936310,-87.651563,POINT (-87.6515625922 41.9363101308),20,2019-01-26
1,c2fc499b015dcbc491cd5bbdc7b9e0cfcff3cd3e,2019-01-28 10:45:00,01/28/2019 11:00:00 AM,959.0,4.0,NaN,NaN,3.0,14.0,10.0,...,False,1.0,41.965812,-87.655879,POINT (-87.6558787862 41.96581197),41.968069,-87.721559,POINT (-87.7215590627 41.968069),10,2019-01-28
2,c2fc49b3505ccf5ef937e6b772e700710d2d271d,2019-02-10 13:30:00,02/10/2019 01:45:00 PM,496.0,1.5,1.703106e+10,1.703107e+10,6.0,7.0,5.0,...,False,1.0,41.936237,-87.656412,POINT (-87.6564115308 41.9362371791),41.921778,-87.651062,POINT (-87.6510618838 41.9217781876),13,2019-02-10
3,c2fc49cc92978244e8a5c2dfde42b3f42c4c1381,2019-02-28 05:00:00,02/28/2019 05:00:00 AM,430.0,1.1,1.703108e+10,1.703184e+10,8.0,32.0,5.0,...,False,1.0,41.890922,-87.618868,POINT (-87.6188683546 41.8909220259),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),5,2019-02-28
4,c2fc4aab2597d35d8a0a8ccb53d08a93df5e9778,2018-12-18 20:30:00,12/18/2018 08:30:00 PM,693.0,2.0,1.703122e+10,1.703124e+10,22.0,24.0,7.5,...,False,1.0,41.921085,-87.704648,POINT (-87.704648384 41.9210845826),41.906026,-87.675312,POINT (-87.6753116216 41.906025969),20,2018-12-18


In [3]:
from datetime import datetime
ride_info['Trip Start Timestamp'] = [datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p') for x in ride_info['Trip Start Timestamp']]
ride_info['hour'] = ride_info['Trip Start Timestamp'].apply(lambda x: x.hour)
ride_info['date'] = ride_info['Trip Start Timestamp'].apply(lambda x: x.date())
ride_info['origin'] = ride_info['originCounty'].apply(lambda x: fixPad(x))
ride_info['dest'] = ride_info['destinationCounty'].apply(lambda x: fixPad(x))
clean_ride = ride_info[['origin', 'dest', 'date', 'hour', 'Trip ID']]
group_ride = clean_ride.groupby(['origin', 'dest', 'date', 'hour']).count().reset_index()
group_ride['day'] = group_ride['date'].apply(lambda x: x.weekday())

In [2]:
print(len(group_ride))
# print(type(group_ride.dest[1]))
group_ride.head(1)

In [ ]:
# # Figure of total rides by hour
# import matplotlib.pyplot as plt; plt.rcdefaults()
# import numpy as np
# import matplotlib.pyplot as plt
# sample = group_ride.groupby('hour', as_index=False).sum()
# sample
# plt.bar(sample['hour'], sample['Trip ID'], align='center', alpha=0.5)
# plt.ylabel('Ride Counts')
# plt.show()

In [29]:
# group_ride = group_ride.iloc[0:10000]
# len(group_ride)

10000

## **Import POIs**

In [ ]:
# POIs_url = 'https://drive.google.com/open?id=1iUnzRXrgTa5itcO6sLke_YXrKFa5YWVH'
# id = '1iUnzRXrgTa5itcO6sLke_YXrKFa5YWVH' # The shareable link
# downloaded = drive.CreateFile({'id':id}) 
# downloaded.GetContentFile('Filename.csv')  
# poi = pd.read_csv('Filename.csv')
poi = pd.read_csv('Filename.csv')

In [31]:
droplist = list(['Unnamed: 0',
 'geometry',
 'statefp10',
 'name10',
 'commarea_n',
 'namelsad10',
 'commarea',
 'notes',
 'countyfp10',
 'centroid'])
poi = poi.drop(droplist, axis=1)
poi['county'] = poi['geoid10'].apply(lambda x: str(x)[-6:])
poi.head(1)

,geoid10,Area,accounting,airport,amusement_park,aquarium,art_gallery,atm,bakery,bank,bar,beauty_salon,bicycle_store,book_store,bowling_alley,bus_station,cafe,car_dealer,car_rental,car_repair,car_wash,casino,cemetery,church,city_hall,clothing_store,convenience_store,dentist,department_store,doctor,drugstore,electrician,electronics_store,embassy,finance,fire_station,florist,food,funeral_home,furniture_store,...,park,parking,pet_store,pharmacy,physiotherapist,place_of_worship,plumber,police,political,post_office,premise,primary_school,real_estate_agency,restaurant,roofing_contractor,rv_park,school,secondary_school,shoe_store,shopping_mall,spa,stadium,storage,store,subway_station,supermarket,synagogue,taxi_stand,tourist_attraction,train_station,transit_station,travel_agency,university,veterinary_care,zoo,tot_POIs,pub_trans_POIs,dens_tot_POIs,dens_pub_trans_POIs,county
0,17031010100,383167.215833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,21.0,0.0,54.806359,0.0,010100


## **Import Census**

In [ ]:
# Pop_Dem_Data = 'https://drive.google.com/open?id=1CtlrPtKsWvUNVreGawGu7fpg-0h6pGpZ'
id = '1CtlrPtKsWvUNVreGawGu7fpg-0h6pGpZ' # The shareable link
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  
census = pd.read_csv('Filename.csv')
census['Census_tracts'] = census['Census_tracts'].astype(int)
census['Census_tracts1'] = census['Census_tracts'].apply(lambda x: fixPad(x))
census['Census_tracts'] = census['Census_tracts1'].astype(str)
census.drop(['Unnamed: 0', 'Census_tracts1'], axis=1, inplace=True)

In [32]:
census.head(1)

,Census_tracts,Total_car_truck_or_van_carpooled,Total_workers_who_walked,Cash_Median_income,Num_Less_than_10_min_to_work,Num_Used_public_transprotation_excluding_taxicab,Population,Male,Female,White,American_indian_or_alaska_native,Asian,Male18_19,Male_20,Male21,Male_22_24,Male_25_29,Male_30_34,Male35_39,Male_40_44,Male45_49,Male_50_54,Male_55_59,Male_60_61,Male_62_64,Male_65_66,Male_67_69,Male_70_74,FeMale18_19,FeMale_20,FeMale21,FeMale_22_24,FeMale_25_29,FeMale_30_34,FeMale35_39,Male_40_44.1,FeMale45_49,FeMale_50_54,FeMale_55_59,FeMale_60_61,FeMale_62_64,FeMale_65_66,FeMale_67_69,FeMale_70_74,Black
0,881105,155.598206,59.146427,-2.310747e+06,222.727331,182.380006,323.0,3503.0,2891.0,17.0,40.0,101.0,122.0,58.0,16.0,90.0,218.0,107.0,297.0,141.0,238.0,254.0,299.0,126.0,112.0,129.0,47.0,143.0,51.0,14.0,0.0,50.0,241.0,225.0,161.0,141.0,154.0,244.0,382.0,122.0,154.0,113.0,51.0,59.0,3503.0


In [ ]:
census.sort_values(by=['Census_tracts'])

## **Import Weather**

In [ ]:

# weather_url = 'https://drive.google.com/open?id=1Wt-yCli7ez1AgtlOLbEk3omiOJ6eoixC'
id = '1Wt-yCli7ez1AgtlOLbEk3omiOJ6eoixC' # The shareable link
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  
weather = pd.read_csv('Filename.csv')
weather['hour'] = weather['time'].apply(lambda x: formatTime(x))
weather['date1'] = [datetime.strptime(x, '%Y-%m-%d') for x in weather['date']]
weather['date'] = weather['date1'].apply(lambda x: x.date())
weather = weather.drop(['Unnamed: 0', 'time','date1'], axis=1)

In [33]:
weather.head(1)

,date,temp,windspeedMiles,precipMM,humidity,visibility,WindChillF,uvIndex,hour
0,2019-09-01,67,10,0.7,80,28,67,0,0


## **Merge 1-3**

In [ ]:
# df1.merge(df2, left_on='lkey', right_on='rkey')
firstMerge = group_ride.merge(poi, how='left', left_on='origin', right_on='county')
secondMerge = firstMerge.merge(poi, how='left', left_on='dest', right_on='county')

In [35]:
secondMerge.head(1)

,origin,dest,date,hour,Trip ID,day,geoid10_x,Area_x,accounting_x,airport_x,amusement_park_x,aquarium_x,art_gallery_x,atm_x,bakery_x,bank_x,bar_x,beauty_salon_x,bicycle_store_x,book_store_x,bowling_alley_x,bus_station_x,cafe_x,car_dealer_x,car_rental_x,car_repair_x,car_wash_x,casino_x,cemetery_x,church_x,city_hall_x,clothing_store_x,convenience_store_x,dentist_x,department_store_x,doctor_x,drugstore_x,electrician_x,electronics_store_x,embassy_x,...,park_y,parking_y,pet_store_y,pharmacy_y,physiotherapist_y,place_of_worship_y,plumber_y,police_y,political_y,post_office_y,premise_y,primary_school_y,real_estate_agency_y,restaurant_y,roofing_contractor_y,rv_park_y,school_y,secondary_school_y,shoe_store_y,shopping_mall_y,spa_y,stadium_y,storage_y,store_y,subway_station_y,supermarket_y,synagogue_y,taxi_stand_y,tourist_attraction_y,train_station_y,transit_station_y,travel_agency_y,university_y,veterinary_care_y,zoo_y,tot_POIs_y,pub_trans_POIs_y,dens_tot_POIs_y,dens_pub_trans_POIs_y,county_y
0,010100,010100,2019-09-01,14,1,6,17031010100,383167.215833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,21.0,0.0,54.806359,0.0,010100


In [ ]:
thirdMerge = secondMerge.merge(weather, how='left', left_on=['date', 'hour'], right_on=['date', 'hour'])

In [37]:
thirdMerge.head(1)

,origin,dest,date,hour,Trip ID,day,geoid10_x,Area_x,accounting_x,airport_x,amusement_park_x,aquarium_x,art_gallery_x,atm_x,bakery_x,bank_x,bar_x,beauty_salon_x,bicycle_store_x,book_store_x,bowling_alley_x,bus_station_x,cafe_x,car_dealer_x,car_rental_x,car_repair_x,car_wash_x,casino_x,cemetery_x,church_x,city_hall_x,clothing_store_x,convenience_store_x,dentist_x,department_store_x,doctor_x,drugstore_x,electrician_x,electronics_store_x,embassy_x,...,police_y,political_y,post_office_y,premise_y,primary_school_y,real_estate_agency_y,restaurant_y,roofing_contractor_y,rv_park_y,school_y,secondary_school_y,shoe_store_y,shopping_mall_y,spa_y,stadium_y,storage_y,store_y,subway_station_y,supermarket_y,synagogue_y,taxi_stand_y,tourist_attraction_y,train_station_y,transit_station_y,travel_agency_y,university_y,veterinary_care_y,zoo_y,tot_POIs_y,pub_trans_POIs_y,dens_tot_POIs_y,dens_pub_trans_POIs_y,county_y,temp,windspeedMiles,precipMM,humidity,visibility,WindChillF,uvIndex
0,010100,010100,2019-09-01,14,1,6,17031010100,383167.215833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,21.0,0.0,54.806359,0.0,010100,70,5,0.0,74,28,70,5


In [ ]:
# thirdMerge.to_csv('thirdMerge.csv')

## **Merge 4-5**

In [ ]:
# thirdMerge = pd.read_csv('/content/thirdMerge.csv')

In [38]:
print(type(thirdMerge['origin'][1]))
print(type(thirdMerge['dest'][1]))
print(type(census['Census_tracts'][1]))

<class 'str'>
<class 'str'>
<class 'str'>


In [ ]:
fourthMerge = thirdMerge.merge(census, how='left', left_on='origin', right_on='Census_tracts')

In [40]:
fourthMerge.head()

,origin,dest,date,hour,Trip ID,day,geoid10_x,Area_x,accounting_x,airport_x,amusement_park_x,aquarium_x,art_gallery_x,atm_x,bakery_x,bank_x,bar_x,beauty_salon_x,bicycle_store_x,book_store_x,bowling_alley_x,bus_station_x,cafe_x,car_dealer_x,car_rental_x,car_repair_x,car_wash_x,casino_x,cemetery_x,church_x,city_hall_x,clothing_store_x,convenience_store_x,dentist_x,department_store_x,doctor_x,drugstore_x,electrician_x,electronics_store_x,embassy_x,...,Num_Used_public_transprotation_excluding_taxicab,Population,Male,Female,White,American_indian_or_alaska_native,Asian,Male18_19,Male_20,Male21,Male_22_24,Male_25_29,Male_30_34,Male35_39,Male_40_44,Male45_49,Male_50_54,Male_55_59,Male_60_61,Male_62_64,Male_65_66,Male_67_69,Male_70_74,FeMale18_19,FeMale_20,FeMale21,FeMale_22_24,FeMale_25_29,FeMale_30_34,FeMale35_39,Male_40_44.1,FeMale45_49,FeMale_50_54,FeMale_55_59,FeMale_60_61,FeMale_62_64,FeMale_65_66,FeMale_67_69,FeMale_70_74,Black
0,010100,010100,2019-09-01,14,1,6,17031010100,383167.215833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,12.0,497.0,1246.0,1141.0,1.0,42.0,0.0,34.0,12.0,25.0,42.0,94.0,51.0,96.0,61.0,83.0,71.0,86.0,50.0,43.0,24.0,25.0,122.0,34.0,11.0,11.0,23.0,51.0,66.0,65.0,61.0,91.0,84.0,110.0,43.0,69.0,37.0,22.0,47.0,1246.0
1,010100,010100,2019-09-01,14,1,6,17031010100,383167.215833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1014.0,2177.0,2142.0,37.0,0.0,11.0,40.0,11.0,6.0,54.0,95.0,182.0,119.0,99.0,195.0,179.0,166.0,30.0,88.0,45.0,59.0,130.0,42.0,5.0,8.0,72.0,146.0,138.0,87.0,99.0,189.0,195.0,156.0,28.0,58.0,56.0,71.0,127.0,2177.0
2,010100,010100,2019-09-01,14,1,6,17031010100,383167.215833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2.0,218.0,2499.0,2671.0,910.0,24.0,201.0,39.0,0.0,16.0,57.0,165.0,196.0,108.0,165.0,45.0,188.0,69.0,34.0,147.0,18.0,47.0,44.0,17.0,0.0,0.0,83.0,316.0,87.0,282.0,165.0,176.0,134.0,139.0,50.0,96.0,62.0,81.0,89.0,2499.0
3,010100,010100,2019-09-01,14,1,6,17031010100,383167.215833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,381.0,1915.0,2605.0,14.0,0.0,40.0,55.0,7.0,22.0,71.0,130.0,99.0,130.0,77.0,208.0,178.0,79.0,41.0,40.0,23.0,38.0,65.0,85.0,22.0,26.0,208.0,189.0,99.0,63.0,77.0,177.0,106.0,137.0,44.0,83.0,77.0,54.0,108.0,1915.0
4,010100,010100,2019-09-01,14,1,6,17031010100,383167.215833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,4.0,955.0,3068.0,3001.0,23.0,14.0,43.0,60.0,37.0,27.0,126.0,162.0,176.0,257.0,121.0,201.0,210.0,238.0,92.0,110.0,58.0,136.0,119.0,48.0,7.0,40.0,91.0,121.0,168.0,288.0,121.0,151.0,257.0,168.0,154.0,92.0,81.0,95.0,138.0,3068.0


In [ ]:
fifthMerge = fourthMerge.merge(census, how='left', left_on='dest', right_on='Census_tracts')

In [42]:
fifthMerge.head(1)

,origin,dest,date,hour,Trip ID,day,geoid10_x,Area_x,accounting_x,airport_x,amusement_park_x,aquarium_x,art_gallery_x,atm_x,bakery_x,bank_x,bar_x,beauty_salon_x,bicycle_store_x,book_store_x,bowling_alley_x,bus_station_x,cafe_x,car_dealer_x,car_rental_x,car_repair_x,car_wash_x,casino_x,cemetery_x,church_x,city_hall_x,clothing_store_x,convenience_store_x,dentist_x,department_store_x,doctor_x,drugstore_x,electrician_x,electronics_store_x,embassy_x,...,Num_Used_public_transprotation_excluding_taxicab_y,Population_y,Male_y,Female_y,White_y,American_indian_or_alaska_native_y,Asian_y,Male18_19_y,Male_20_y,Male21_y,Male_22_24_y,Male_25_29_y,Male_30_34_y,Male35_39_y,Male_40_44_y,Male45_49_y,Male_50_54_y,Male_55_59_y,Male_60_61_y,Male_62_64_y,Male_65_66_y,Male_67_69_y,Male_70_74_y,FeMale18_19_y,FeMale_20_y,FeMale21_y,FeMale_22_24_y,FeMale_25_29_y,FeMale_30_34_y,FeMale35_39_y,Male_40_44.1_y,FeMale45_49_y,FeMale_50_54_y,FeMale_55_59_y,FeMale_60_61_y,FeMale_62_64_y,FeMale_65_66_y,FeMale_67_69_y,FeMale_70_74_y,Black_y
0,010100,010100,2019-09-01,14,1,6,17031010100,383167.215833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,12.0,497.0,1246.0,1141.0,1.0,42.0,0.0,34.0,12.0,25.0,42.0,94.0,51.0,96.0,61.0,83.0,71.0,86.0,50.0,43.0,24.0,25.0,122.0,34.0,11.0,11.0,23.0,51.0,66.0,65.0,61.0,91.0,84.0,110.0,43.0,69.0,37.0,22.0,47.0,1246.0


In [ ]:
# fifthMerge.columns.values
dlist = ['geoid10_x', 'geoid10_y', 'county_x', 'county_y', 'Census_tracts_x', 'Census_tracts_y']
fifthMerge.drop(dlist, axis=1, inplace=True)

## Other

In [ ]:
fifthMerge.to_csv('tb_masterWithPOIandDemographics.csv')